## Endpoint execution

This Jupyter Notebook simplifies interaction with the key endpoints of the Capture The Flag platform, offering a user-friendly environment to test and utilize its functionalities. It bridges the gap between backend mechanics and practical applications, making it a valuable resource for developers and enthusiasts.

### Initial imports and configurations

In [19]:
import shutil
import requests
import urllib3
import os
import shutil
import json
import time
from dotenv import load_dotenv

# Login credentials from .env
load_dotenv()
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')

s = requests.session()

# Needed locally only
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
s.verify = False

# Mandatory configuration
keycloak_host = "https://ctf.jacopomauro.com/keycloak"
deployer_host = "https://deployer.ctf.jacopomauro.com"

# Paths to the challenge files
src_folder = "./challenges/challenge-xss/"
src_folder_location      = os.path.join(src_folder, "src")
solution_folder_location = os.path.join(src_folder, "solution")
challenge_yaml_location  = os.path.join(src_folder, "challenge.yml")
handout_folder_location  = os.path.join(src_folder, "handout")

### Get access token
Access token is valid for 5 minutes.
It may be that the operation you are doing will take more than 5 minutes and in this case you have to rerun this code.

In [79]:
def refresh_access_token():
    r = s.post(
        f"{keycloak_host}/realms/ctf/protocol/openid-connect/token/", 
        data={
            "client_id":"deployer", 
            "username": username, 
            "password": password, 
            "grant_type": "password", 
            "scope": "openid"
        }, 
        timeout=20
    )
    print("refreshed access token:", r.status_code, r.content)
    r.raise_for_status()

    s.headers = {"Authorization": "Bearer " + r.json().get("access_token")}

refresh_access_token()

login: 200 b'{"access_token":"eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICItenNFMzl5SzFYMkJlMnlMNkZMU0ZVcDNiQmNId3RXM2RsN1dlNHk0eGl3In0.eyJleHAiOjE3NDc2OTI3MDksImlhdCI6MTc0NzY5MjQwOSwianRpIjoiYzFkMmM0OGItMGM2Mi00MTIzLWE2ZGItZTQ2YWE2ZmI5MWQxIiwiaXNzIjoiaHR0cHM6Ly9jdGYuamFjb3BvbWF1cm8uY29tL2tleWNsb2FrL3JlYWxtcy9jdGYiLCJhdWQiOlsiY3RmZCIsInN0ZXAiLCJhY2NvdW50Il0sInN1YiI6ImEyYTcxYWMzLThjNDgtNDgxZi05M2M4LTRhYjgyNDIyNGNiMyIsInR5cCI6IkJlYXJlciIsImF6cCI6ImRlcGxveWVyIiwic2lkIjoiYWRmNDhkNmMtMmM3Yy00Yjc5LThjYTctOTlkMDUzNDU4MTRkIiwiYWNyIjoiMSIsImFsbG93ZWQtb3JpZ2lucyI6WyIiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImRlZmF1bHQtcm9sZXMtY3RmIiwib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImN0ZmQiOnsicm9sZXMiOlsidXNlciJdfSwic3RlcCI6eyJyb2xlcyI6WyJiYXN0aW9uIl19LCJkZXBsb3llciI6eyJyb2xlcyI6WyJkZXZlbG9wZXIiXX0sImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIGVtYWlsIHByb2ZpbGUiLCJlbWFpbF92ZXJpZmllZCI6Zm

### Post challenge to deployer service

Submit the challenge to the CTF platform.

In [21]:
files = []
files.append(("upload[]", open(challenge_yaml_location, "rb")))

# only zip & queue the challenge bundle if its folder exists
if os.path.isdir(src_folder_location):
    shutil.make_archive("challenge", "zip", src_folder_location)
    files.append(("upload[]", open("challenge.zip", "rb")))

# only zip & queue the solution bundle if its folder exists
if os.path.isdir(solution_folder_location):
    shutil.make_archive("solution", "zip", solution_folder_location)
    files.append(("upload[]", open("solution.zip", "rb")))

# only zip & queue the handout bundle if its folder exists
if os.path.isdir(handout_folder_location):
    shutil.make_archive("handout", "zip", handout_folder_location)
    files.append(("upload[]", open("handout.zip", "rb")))

# if there's nothing to send, bail out early
if not files:
    print("No folders found to package — nothing to upload.")
else:
    r = s.post(
        deployer_host + "/challenges",
        files=files,
        timeout=20
    )
    # close all file handles
    for _, fh in files:
        fh.close()

    print("add challenge:", r.status_code, r.content)
    r.raise_for_status()
    challenge_id = r.json().get("challengeid")


add challenge: 200 b'{"challengeid":"13d1e533-4f58-451e-9215-8231d49e00e8"}'


### Start the Challenge using the API

To start the challenge use the start API.
Note that running the challenge cat take minutes (the system has to run the VM, run the docker-compose, build the docker images, run the docker images, ...).
Note that you should be able to run only one challenge at a time. For stopping, see below.

Note that in case of problems with a certificate in Chrome you can try typing “thisisunsafe”

In [22]:
r = s.post(deployer_host + "/challenges/" + challenge_id + "/start", timeout=20)
print("start challenge:", r.status_code, r.content)
r.raise_for_status()

start challenge: 200 b'{"url":"64408aac-5c86-4bdd.deployer.ctf.jacopomauro.com","secondslseft":1200,"started":true,"verified":false}'


### Start Test using the API

This API call will run the solution command in a VM. Note that is better to execute this command after the challenge is running.
You can verify this by using the domain provided by starting the challenge and invoking the health check.
As for the challenge, running the validation can take quite a long time (the system has to run the VM, build the docker images, run it)

In [12]:
r = s.post(deployer_host + f"/solutions/{challenge_id}/start", timeout=20)
print("start test:", r.status_code, r.content)
r.raise_for_status()

start test: 200 b'{"url":"d10b5e0a-9414-4a8e.deployer.ctf.jacopomauro.com","secondslseft":1200,"started":true,"verified":false}'


### Check Challenge Status

This allows to check the status of a submitted challenge.
In particular, you can understand if it has been validated or not.

```json
{"ready":"","secondsleft":"","started":"","url":"","verified":""}
```

In [23]:
r = s.get(deployer_host + "/challenges/" + challenge_id + "/status", timeout=20)
print("logs:", r.status_code, r.content)
r.raise_for_status()

logs: 200 b'{"ready":false,"secondsleft":1196,"started":true,"url":"64408aac-5c86-4bdd.deployer.ctf.jacopomauro.com","verified":false}'


### Check If Challenge Is Ready

In [ ]:
while True:
    r = s.get(deployer_host + "/challenges/" + challenge_id + "/status", timeout=20)
    if r.ok:
        print("logs:", r.status_code, r.content)
        if json.loads(r.content)["ready"]:
            print("I'm ready :D")
            break
    elif r.status_code == 401:
        refresh_access_token()
    else:
        r.raise_for_status()
    time.sleep(30)

logs: 200 b'{"ready":false,"secondsleft":1194,"started":true,"url":"64408aac-5c86-4bdd.deployer.ctf.jacopomauro.com","verified":false}'
logs: 200 b'{"ready":false,"secondsleft":1164,"started":true,"url":"64408aac-5c86-4bdd.deployer.ctf.jacopomauro.com","verified":false}'
logs: 200 b'{"ready":false,"secondsleft":1134,"started":true,"url":"64408aac-5c86-4bdd.deployer.ctf.jacopomauro.com","verified":false}'
logs: 200 b'{"ready":false,"secondsleft":1104,"started":true,"url":"64408aac-5c86-4bdd.deployer.ctf.jacopomauro.com","verified":false}'
logs: 200 b'{"ready":false,"secondsleft":1073,"started":true,"url":"64408aac-5c86-4bdd.deployer.ctf.jacopomauro.com","verified":false}'
logs: 200 b'{"ready":false,"secondsleft":1043,"started":true,"url":"64408aac-5c86-4bdd.deployer.ctf.jacopomauro.com","verified":false}'
logs: 200 b'{"ready":false,"secondsleft":1013,"started":true,"url":"64408aac-5c86-4bdd.deployer.ctf.jacopomauro.com","verified":false}'
logs: 200 b'{"ready":false,"secondsleft":983,"st

### Stop the Test using the API

The test should stop automatically in case of problems after a certain amount of time.
Since it is possible only to execute a test at a time, you can stop it with this API in case of problems.

In [17]:
r = s.post(deployer_host + "/solutions/" + challenge_id + "/stop", timeout=20)
print("stop test:", r.status_code, r.content)
r.raise_for_status()

stop test: 404 b'{"message":"Test instance not running"}'


HTTPError: 404 Client Error: Not Found for url: https://deployer.ctf.jacopomauro.com/solutions/8c0da34b-f234-4f0a-8d28-ce0286b7f999/stop

### Stop the Challenge using the API

When finishing testing (manually or automatically), use this API to stop the challenge.

In [18]:
r = s.post(deployer_host + "/challenges/" + challenge_id + "/stop", timeout=20)
print("stop challenge:", r.status_code, r.content)
r.raise_for_status()

stop challenge: 200 b'{"message":"Stopping challenge"}'


### Get Logs of the Challenge Instance

Use this API to extract the logs for running the challenge.


In [77]:
import time
while True:
    r = s.get(deployer_host + "/challenges/" + challenge_id + "/logs", timeout=20)
    if r.ok:
        print(f"Successfully fetched logs (status {r.status_code}):")
        print(r.status_code)
        print(r.content.decode())
    else:
        print(f"Failed to fetch logs (status {r.status_code} {r.reason})")
        print("Response body:", r.text)
        if r.status_code == 401:
            refresh_access_token()
            continue
        r.raise_for_status()
    time.sleep(10)

Successfully fetched logs (status 200):
200

SYSLINUX 6.04 6.04-pre1 Copyright (C) 1994-2015 H. Peter Anvin et al
e%@)0(BAlpine will be booted automatically in 1 seconds.                                                  Loading vmlinuz-virt... ok
Loading initramfs-virt...ok
[    0.000000] Linux version 6.6.56-0-virt (buildozer@build-edge-x86_64) (gcc (Alpine 14.2.0) 14.2.0, GNU ld (GNU Binutils) 2.43.1) #1-Alpine SMP PREEMPT_DYNAMIC 2024-10-10 16:45:57
[    0.000000] Command line: BOOT_IMAGE=vmlinuz-virt root=LABEL=/ modules=sd-mod,usb-storage,ext4 console=ttyS0,115200n8 console=tty0 initrd=initramfs-virt
[    0.000000] BIOS-provided physical RAM map:
[    0.000000] BIOS-e820: [mem 0x0000000000000000-0x000000000009fbff] usable
[    0.000000] BIOS-e820: [mem 0x000000000009fc00-0x000000000009ffff] reserved
[    0.000000] BIOS-e820: [mem 0x00000000000f0000-0x00000000000fffff] reserved
[    0.000000] BIOS-e820: [mem 0x0000000000100000-0x000000007a1dcfff] usable
[    0.000000] BIOS-e820

KeyboardInterrupt: 

### Get Logs of the Solution Instance

Note that when the instance has ended the logs are not available anymore.

In [15]:
import time
i = 0
while True:
    r = s.get(deployer_host + "/solutions/" + challenge_id + "/logs", timeout=20)
    if r.ok:
        print(r.content.decode())
    else:
        print(f"Failed to fetch logs (status {r.status_code} {r.reason})")
        print("Response body:", r.text)
        if r.status_code == 401:
            refresh_access_token()
            continue
        r.raise_for_status()
    time.sleep(5)

Failed to fetch logs (status 401 Unauthorized)
Response body: 
login: 200 b'{"access_token":"eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICItenNFMzl5SzFYMkJlMnlMNkZMU0ZVcDNiQmNId3RXM2RsN1dlNHk0eGl3In0.eyJleHAiOjE3NDc2OTI2MTEsImlhdCI6MTc0NzY5MjMxMSwianRpIjoiYTFjNTE1Y2MtNjFhOC00ZGZjLTliNDUtNWYxYThhZWVmMDU1IiwiaXNzIjoiaHR0cHM6Ly9jdGYuamFjb3BvbWF1cm8uY29tL2tleWNsb2FrL3JlYWxtcy9jdGYiLCJhdWQiOlsiY3RmZCIsInN0ZXAiLCJhY2NvdW50Il0sInN1YiI6ImEyYTcxYWMzLThjNDgtNDgxZi05M2M4LTRhYjgyNDIyNGNiMyIsInR5cCI6IkJlYXJlciIsImF6cCI6ImRlcGxveWVyIiwic2lkIjoiMTg0N2EyNjUtZGFiZS00ZjE3LWEyYzYtZDU0NWU0MzQzYjQ3IiwiYWNyIjoiMSIsImFsbG93ZWQtb3JpZ2lucyI6WyIiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImRlZmF1bHQtcm9sZXMtY3RmIiwib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImN0ZmQiOnsicm9sZXMiOlsidXNlciJdfSwic3RlcCI6eyJyb2xlcyI6WyJiYXN0aW9uIl19LCJkZXBsb3llciI6eyJyb2xlcyI6WyJkZXZlbG9wZXIiXX0sImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInN

KeyboardInterrupt: 

### SSH command

The following SSH command demonstrates how to connect to a CTF Challenge using SSH, leveraging the Bastion as a jump proxy while enabling port forwarding from the container to your local machine. This method allows access to a wider range of ports beyond those directly exposed by the platform itself.

```bash
ssh <username>@ssh.<subdomain> -p 8022 -J bastion@ctf.jacopomauro.com:443 -L <local-port>:<service:port>
sshpass -p 'password' ssh -o StrictHostKeyChecking=accept-new -p 8022 <user>@ssh.challenge-<subdomain> -J bastion@ctf.jacopomauro.com:443 -L <local-port>:<service:port> -N -f
```

## Get list of challenges

In [ ]:
r = s.get(deployer_host + "/challenges", timeout=20)
if r.ok:
    print(f"Successfully fetched challenges (status {r.status_code}):")
    print(r.content.decode())
    challs = r.json()["challenges"]
    for chall in challs:
        print(chall["id"])
else:
    print(f"Failed to fetch challenges (status {r.status_code} {r.reason})")
    print("Response body:", r.text)
    r.raise_for_status()

Successfully fetched challenges (status 200):
{"challenges":[{"id":"362b3c96-cba7-46a0-a461-dc4367666aba","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false,"ctfd_id":{"Int64":0,"Valid":false},"verified":false},{"id":"8ca6cff1-9f43-46af-b98d-f92d717b25f3","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false,"ctfd_id":{"Int64":0,"Valid":false},"verified":true},{"id":"22b90ae8-6138-4f55-8220-bb280f884911","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false,"ctfd_id":{"Int64":0,"Valid":false},"verified":false},{"id":"fde858d3-34fa-41f2-9804-c4af5193d466","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false,"ctfd_id":{"Int64":0,"Valid":false},"verified":false},{"id":"ea1c94c7-b57d-4146-a3e1-a0764dcabcbe","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false,"ctfd_id":{"Int64":0,"Valid":false},"verified":true},{"id":"af0f8206-37d1-450f-89d4-c3898186e601","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false

## Stop All Challenges

In [ ]:
r = s.get(deployer_host + "/challenges", timeout=20)
if r.ok:
    print(f"Successfully fetched challenges (status {r.status_code}):")
    print(r.content.decode())
    challs = r.json()["challenges"]
    for chall in challs:
        id = chall["id"]
        r = s.post(deployer_host + "/challenges/" + id + "/stop", timeout=20)
        print("stop challenge:", r.status_code, r.content)
        try:
            r.raise_for_status()
        except:
            pass
else:
    print(f"Failed to fetch challenges (status {r.status_code} {r.reason})")
    print("Response body:", r.text)
    r.raise_for_status()

Successfully fetched challenges (status 200):
{"challenges":[{"id":"362b3c96-cba7-46a0-a461-dc4367666aba","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false,"ctfd_id":{"Int64":0,"Valid":false},"verified":false},{"id":"8ca6cff1-9f43-46af-b98d-f92d717b25f3","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false,"ctfd_id":{"Int64":0,"Valid":false},"verified":true},{"id":"22b90ae8-6138-4f55-8220-bb280f884911","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false,"ctfd_id":{"Int64":0,"Valid":false},"verified":false},{"id":"fde858d3-34fa-41f2-9804-c4af5193d466","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false,"ctfd_id":{"Int64":0,"Valid":false},"verified":false},{"id":"ea1c94c7-b57d-4146-a3e1-a0764dcabcbe","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false,"ctfd_id":{"Int64":0,"Valid":false},"verified":true},{"id":"af0f8206-37d1-450f-89d4-c3898186e601","user_id":"72486f64-8c4f-48ff-b986-81b1b9e0882d","published":false

## Delete challenge

In [119]:
challenge_id = "284e449f-5cc0-4db4-8cd5-570db04acd72"
r = s.delete(deployer_host + "/challenges/" + challenge_id, timeout=20)
if r.ok:
    print(f"Successfully deleted challenge (status {r.status_code}):")
    print(r.content.decode())
else:
    print(f"Failed to delete challenge (status {r.status_code} {r.reason})")
    print("Response body:", r.text)
    r.raise_for_status()

Successfully deleted challenge (status 200):
{"challengeid":"284e449f-5cc0-4db4-8cd5-570db04acd72"}
